In [1]:
# autograd_tensor.py
# Minimal numpy-based autograd engine with broadcasting and batch support.
# Ops: +, -, neg, *, **, sqrt, @, /, T, reshape, exp, log, mean, gather, sum
# sum and gather support axis/dim. T swaps the last two dims for ND tensors.

from __future__ import annotations
import numpy as np
from typing import Optional, Tuple, Union, Iterable

ArrayLike = Union[np.ndarray, float, int]

def _as_array(x: ArrayLike, dtype=np.float32) -> np.ndarray:
    if isinstance(x, np.ndarray):
        return x.astype(dtype, copy=False)
    return np.array(x, dtype=dtype)

def _normalize_axes(axis, ndim: int) -> Optional[Tuple[int, ...]]:
    if axis is None:
        return None
    if isinstance(axis, (list, tuple)):
        axes = tuple((a + ndim) if a < 0 else a for a in axis)
        return tuple(sorted(axes))
    a = axis + ndim if axis < 0 else axis
    return (a,)

def _sum_to_shape(x: np.ndarray, shape: Tuple[int, ...]) -> np.ndarray:
    """Sum x over broadcasted axes so that it matches 'shape'."""
    if x.shape == shape:
        return x
    # Remove leading broadcasted dims
    while x.ndim > len(shape):
        x = x.sum(axis=0)
    # Sum over axes where original had size 1
    for i, (sx, so) in enumerate(zip(x.shape, shape)):
        if so == 1 and sx != 1:
            x = x.sum(axis=i, keepdims=True)
    return x.reshape(shape)

def _expand_grad_to_shape(g: np.ndarray, in_shape: Tuple[int, ...],
                          axis: Optional[Tuple[int, ...]], keepdims: bool) -> np.ndarray:
    """Expand reduced gradient g back to input shape for sum/mean backward."""
    if axis is None:
        return np.broadcast_to(g, in_shape)
    axes = axis
    # Re-insert reduced dims if keepdims=False
    if not keepdims:
        # Insert dims in sorted order to keep positions right
        for ax in axes:
            g = np.expand_dims(g, ax)
    return np.broadcast_to(g, in_shape)

def _swap_last2(x: np.ndarray) -> np.ndarray:
    if x.ndim < 2:
        return x  # no-op for 0D/1D
    return np.swapaxes(x, -1, -2)

def _scatter_add_along_axis(base: np.ndarray, axis: int,
                            indices: np.ndarray, updates: np.ndarray) -> None:
    """
    Scatter-add 'updates' into 'base' along 'axis' using 'indices' like the inverse of take_along_axis.
    Modifies 'base' in-place.
    """
    axis = axis if axis >= 0 else axis + base.ndim
    base_m = np.moveaxis(base, axis, 0)
    idx_m = np.moveaxis(indices, axis, 0)
    upd_m = np.moveaxis(updates, axis, 0)

    # Build coordinates for advanced indexing
    # idx_m shape: (K, s1, s2, ...); grid gives coordinates for remaining axes
    grid = np.indices(idx_m.shape)
    # grid[0] are positions along the axis of 'updates', which we don't need for base (replaced by idx_m)
    coords = [idx_m] + [grid[i] for i in range(1, grid.shape[0])]
    np.add.at(base_m, tuple(coords), upd_m)

def _to_tensor(x: ArrayLike, requires_grad: bool = False) -> "Tensor":
    return x if isinstance(x, Tensor) else Tensor(x, requires_grad=requires_grad)

class Tensor:
    def __init__(self, data: ArrayLike, requires_grad: bool = False, name: Optional[str] = None):
        if isinstance(data, Tensor):
            data = data.data
        self.data: np.ndarray = _as_array(data)
        self.grad: Optional[np.ndarray] = None
        self.requires_grad: bool = requires_grad
        self._backward = lambda: None
        self._prev: set[Tensor] = set()
        self.name = name  # optional debug label

    # ----------- Utility -----------
    @property
    def shape(self) -> Tuple[int, ...]:
        return self.data.shape

    def zero_grad(self):
        """Recursively zero grads for all nodes reachable from this Tensor."""
        visited = set()
        def visit(t: Tensor):
            if t in visited:
                return
            visited.add(t)
            t.grad = None
            for p in t._prev:
                visit(p)
        visit(self)

    def detach(self) -> "Tensor":
        out = Tensor(self.data.copy(), requires_grad=False, name=f"{self.name}_detached" if self.name else None)
        return out

    def numpy(self) -> np.ndarray:
        return self.data

    def item(self) -> float:
        return float(self.data.item())

    # ----------- Autograd core -----------
    def backward(self, grad: Optional[ArrayLike] = None):
        if grad is None:
            if self.data.size != 1:
                raise ValueError("grad must be specified for non-scalar outputs")
            grad = np.ones_like(self.data, dtype=np.float32)
        else:
            grad = _as_array(grad)

        topo: list[Tensor] = []
        visited: set[Tensor] = set()

        def build(v: Tensor):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build(child)
                topo.append(v)

        build(self)
        self.grad = grad
        for v in reversed(topo):
            v._backward()

    # ----------- Overloads -----------
    def __add__(self, other: ArrayLike) -> "Tensor":
        other = _to_tensor(other)
        data = self.data + other.data
        out = Tensor(data, requires_grad=self.requires_grad or other.requires_grad)

        def _backward():
            if out.grad is None:
                return
            if self.requires_grad:
                g = _sum_to_shape(out.grad, self.shape)
                self.grad = g if self.grad is None else self.grad + g
            if other.requires_grad:
                g = _sum_to_shape(out.grad, other.shape)
                other.grad = g if other.grad is None else other.grad + g

        out._prev = {self, other}
        out._backward = _backward
        return out
    __radd__ = __add__

    def __neg__(self) -> "Tensor":
        data = -self.data
        out = Tensor(data, requires_grad=self.requires_grad)

        def _backward():
            if out.grad is None or not self.requires_grad:
                return
            g = -out.grad
            self.grad = g if self.grad is None else self.grad + g

        out._prev = {self}
        out._backward = _backward
        return out

    def __sub__(self, other: ArrayLike) -> "Tensor":
        return self + (-_to_tensor(other))
    def __rsub__(self, other: ArrayLike) -> "Tensor":
        return _to_tensor(other) + (-self)

    def __mul__(self, other: ArrayLike) -> "Tensor":
        other = _to_tensor(other)
        data = self.data * other.data
        out = Tensor(data, requires_grad=self.requires_grad or other.requires_grad)

        def _backward():
            if out.grad is None:
                return
            if self.requires_grad:
                g = out.grad * other.data
                g = _sum_to_shape(g, self.shape)
                self.grad = g if self.grad is None else self.grad + g
            if other.requires_grad:
                g = out.grad * self.data
                g = _sum_to_shape(g, other.shape)
                other.grad = g if other.grad is None else other.grad + g

        out._prev = {self, other}
        out._backward = _backward
        return out
    def __rmul__(self, other: ArrayLike) -> "Tensor":
        return self * other

    def __truediv__(self, other: ArrayLike) -> "Tensor":
        other = _to_tensor(other)
        data = self.data / other.data
        out = Tensor(data, requires_grad=self.requires_grad or other.requires_grad)

        def _backward():
            if out.grad is None:
                return
            if self.requires_grad:
                g = out.grad / other.data
                g = _sum_to_shape(g, self.shape)
                self.grad = g if self.grad is None else self.grad + g
            if other.requires_grad:
                g = -out.grad * self.data / (other.data ** 2)
                g = _sum_to_shape(g, other.shape)
                other.grad = g if other.grad is None else other.grad + g

        out._prev = {self, other}
        out._backward = _backward
        return out
    def __rtruediv__(self, other: ArrayLike) -> "Tensor":
        other = _to_tensor(other)
        return other / self

    def __pow__(self, other: ArrayLike) -> "Tensor":
        # supports scalar or Tensor exponent
        if isinstance(other, Tensor):
            base = self
            exp = other
            data = np.power(base.data, exp.data)
            out = Tensor(data, requires_grad=base.requires_grad or exp.requires_grad)

            def _backward():
                if out.grad is None:
                    return
                # d/dx x^y = x^y * y / x ; d/dy x^y = x^y * log(x)
                safe_base = np.where(base.data == 0, 1.0, base.data)
                if base.requires_grad:
                    g = out.grad * data * (exp.data / safe_base)
                    g = _sum_to_shape(g, base.shape)
                    base.grad = g if base.grad is None else base.grad + g
                if exp.requires_grad:
                    # log defined only for base > 0; clip to avoid nan in toy engine
                    g = out.grad * data * np.log(np.clip(base.data, 1e-12, None))
                    g = _sum_to_shape(g, exp.shape)
                    exp.grad = g if exp.grad is None else exp.grad + g

            out._prev = {base, exp}
            out._backward = _backward
            return out
        else:
            exp = float(other)
            data = self.data ** exp
            out = Tensor(data, requires_grad=self.requires_grad)

            def _backward():
                if out.grad is None or not self.requires_grad:
                    return
                g = out.grad * (exp * (self.data ** (exp - 1.0)))
                g = _sum_to_shape(g, self.shape)
                self.grad = g if self.grad is None else self.grad + g

            out._prev = {self}
            out._backward = _backward
            return out
    def __rpow__(self, other: ArrayLike) -> "Tensor":
        other = _to_tensor(other)
        return other ** self

    def __matmul__(self, other: ArrayLike) -> "Tensor":
        other = _to_tensor(other)
        data = np.matmul(self.data, other.data)
        out = Tensor(data, requires_grad=self.requires_grad or other.requires_grad)

        def _backward():
            if out.grad is None:
                return
            if self.requires_grad:
                g = np.matmul(out.grad, _swap_last2(other.data))
                g = _sum_to_shape(g, self.shape)
                self.grad = g if self.grad is None else self.grad + g
            if other.requires_grad:
                g = np.matmul(_swap_last2(self.data), out.grad)
                g = _sum_to_shape(g, other.shape)
                other.grad = g if other.grad is None else other.grad + g

        out._prev = {self, other}
        out._backward = _backward
        return out

    # ----------- Functions -----------
    @property
    def T(self) -> "Tensor":
        data = _swap_last2(self.data)
        out = Tensor(data, requires_grad=self.requires_grad)

        def _backward():
            if out.grad is None or not self.requires_grad:
                return
            g = _swap_last2(out.grad)
            self.grad = g if self.grad is None else self.grad + g

        out._prev = {self}
        out._backward = _backward
        return out

    def reshape(self, *shape: int) -> "Tensor":
        # Allow reshape((...)) or reshape(a,b,c)
        new_shape = shape[0] if len(shape) == 1 and isinstance(shape[0], (tuple, list)) else shape
        data = self.data.reshape(*new_shape)
        out = Tensor(data, requires_grad=self.requires_grad)
        old_shape = self.shape

        def _backward():
            if out.grad is None or not self.requires_grad:
                return
            g = out.grad.reshape(old_shape)
            self.grad = g if self.grad is None else self.grad + g

        out._prev = {self}
        out._backward = _backward
        return out

    def flatten(self, start_dim: int = 0, end_dim: int = -1) -> "Tensor":
        nd = self.data.ndim
        if end_dim < 0:
            end_dim += nd
        if not (0 <= start_dim <= end_dim < nd):
            raise ValueError("Invalid flatten dims")
        pre = self.shape[:start_dim]
        mid = int(np.prod(self.shape[start_dim:end_dim+1], dtype=np.int64))
        post = self.shape[end_dim+1:]
        return self.reshape(*(pre + (mid,) + post))

    def exp(self) -> "Tensor":
        data = np.exp(self.data)
        out = Tensor(data, requires_grad=self.requires_grad)

        def _backward():
            if out.grad is None or not self.requires_grad:
                return
            g = out.grad * out.data  # derivative of exp is exp(x)
            self.grad = g if self.grad is None else self.grad + g

        out._prev = {self}
        out._backward = _backward
        return out

    def log(self) -> "Tensor":
        data = np.log(self.data)
        out = Tensor(data, requires_grad=self.requires_grad)

        def _backward():
            if out.grad is None or not self.requires_grad:
                return
            g = out.grad / self.data
            self.grad = g if self.grad is None else self.grad + g

        out._prev = {self}
        out._backward = _backward
        return out

    def sqrt(self) -> "Tensor":
        data = np.sqrt(self.data)
        out = Tensor(data, requires_grad=self.requires_grad)

        def _backward():
            if out.grad is None or not self.requires_grad:
                return
            g = out.grad * (0.5 / np.sqrt(self.data))
            self.grad = g if self.grad is None else self.grad + g

        out._prev = {self}
        out._backward = _backward
        return out

    def sum(self, axis: Optional[Union[int, Iterable[int]]] = None,
            dim: Optional[Union[int, Iterable[int]]] = None,
            keepdims: bool = False) -> "Tensor":
        ax = dim if dim is not None else axis
        ax_tuple = _normalize_axes(ax, self.data.ndim)
        data = self.data.sum(axis=ax_tuple, keepdims=keepdims)
        out = Tensor(data, requires_grad=self.requires_grad)
        in_shape = self.shape

        def _backward():
            if out.grad is None or not self.requires_grad:
                return
            g = _expand_grad_to_shape(out.grad, in_shape, ax_tuple, keepdims)
            self.grad = g if self.grad is None else self.grad + g

        out._prev = {self}
        out._backward = _backward
        return out

    def mean(self, axis: Optional[Union[int, Iterable[int]]] = None,
             dim: Optional[Union[int, Iterable[int]]] = None,
             keepdims: bool = False) -> "Tensor":
        ax = dim if dim is not None else axis
        ax_tuple = _normalize_axes(ax, self.data.ndim)
        data = self.data.mean(axis=ax_tuple, keepdims=keepdims)
        out = Tensor(data, requires_grad=self.requires_grad)
        in_shape = self.shape

        # number of elements reduced
        if ax_tuple is None:
            N = self.data.size
        else:
            N = 1
            for a in ax_tuple:
                N *= self.data.shape[a]
        N = float(N)

        def _backward():
            if out.grad is None or not self.requires_grad:
                return
            g = out.grad / N
            g = _expand_grad_to_shape(g, in_shape, ax_tuple, keepdims)
            self.grad = g if self.grad is None else self.grad + g

        out._prev = {self}
        out._backward = _backward
        return out

    def gather(self, indices: ArrayLike,
               axis: Optional[int] = None,
               dim: Optional[int] = None) -> "Tensor":
        """
        Gather values along an axis using integer indices (like torch.gather).
        - indices: int array/Tensor. Output shape == indices.shape.
        - axis/dim: axis to gather from (default 0). Supports negative axes.
        Note: indices are treated as non-differentiable.
        """
        ax = dim if dim is not None else (axis if axis is not None else 0)
        idx = indices.data.astype(np.int64) if isinstance(indices, Tensor) else np.asarray(indices, dtype=np.int64)
        data = np.take_along_axis(self.data, idx, axis=ax)
        out = Tensor(data, requires_grad=self.requires_grad)

        def _backward():
            if out.grad is None or not self.requires_grad:
                return
            g_in = np.zeros_like(self.data, dtype=np.float32)
            _scatter_add_along_axis(g_in, ax, idx, out.grad)
            self.grad = g_in if self.grad is None else self.grad + g_in

        out._prev = {self}  # indices are non-differentiable
        out._backward = _backward
        return out

    # ----------- Pretty print -----------
    def __repr__(self) -> str:
        rg = " req_grad" if self.requires_grad else ""
        return f"Tensor(shape={self.data.shape},{rg}, dtype={self.data.dtype}{', name='+self.name if self.name else ''})"

In [6]:
if __name__ == "__main__":
    # Broadcasting add/mul with batch
    x = Tensor(np.random.randn(4, 3, 5), requires_grad=True, name="x")
    b = Tensor(np.random.randn(5), requires_grad=True, name="b")
    y = (x + b).mul = x + b
    z = ((x * 2.0) + y).mean()
    z.backward()
    print("x.grad shape:", x.grad.shape)   # (4,3,5)
    print("b.grad shape:", b.grad.shape)   # (5,)

    # Batched matmul
    a = Tensor(np.random.randn(8, 10, 16), requires_grad=True, name="a")
    w = Tensor(np.random.randn( 16, 32), requires_grad=True, name="w")
    out = a @ w
    print(out.shape)
    out.backward()
    print("a.grad shape:", a.grad.shape)   # (8,10,16)
    print("w.grad shape:", w.grad.shape)   # (8,16,32)

    # Sum and mean over multiple axes
    u = Tensor(np.random.randn(2, 3, 4), requires_grad=True, name="u")
    s = u.sum(axis=(0, 2))
    s.backward(np.ones_like(s.data))
    print("u.grad shape:", u.grad.shape)   # (2,3,4)

    u.zero_grad()
    m = u.mean(dim=1, keepdims=True)
    m.backward(np.ones_like(m.data))
    print("u.grad shape (mean):", u.grad.shape)  # (2,3,4)

    # Gather along axis (like selecting token embeddings)
    emb = Tensor(np.random.randn(4, 7, 16), requires_grad=True, name="emb")  # (B, V, D)
    idx = np.random.randint(0, 7, size=(4, 5, 16))  # gather along axis=1 -> output (4,5,16)
    picked = emb.gather(idx, axis=1)
    loss = picked.sum()
    loss.backward()
    print("emb.grad shape:", emb.grad.shape)  # (4,7,16)

    # Transpose last two dims
    M = Tensor(np.random.randn(3, 5, 7), requires_grad=True, name="M")
    Mt = M.T
    v = (M @ Mt).sum()
    v.backward()
    print("M.grad shape:", M.grad.shape)  # (3,5,7)

    # Reshape/flatten
    t = Tensor(np.random.randn(2, 3, 4), requires_grad=True)
    r = t.flatten(1, 2)  # -> (2, 12)
    (r.sum()).backward()
    print("t.grad shape:", t.grad.shape)  # (2,3,4)

x.grad shape: (4, 3, 5)
b.grad shape: (5,)
(8, 10, 32)


ValueError: grad must be specified for non-scalar outputs

# Chatgpt


In [52]:
# Autograd Tensor implementation for batch/matrix/3D data
# - Supports: +, -, neg, *, **, sqrt, @, /, T, reshape, exp, log, mean, gather, sum
# - sum and gather accept axis/dim arguments
# - Backprop handles broadcasting and batched matmul
# - Includes small tests demonstrating gradients on batch tensors
import numpy as np

# Utility helpers
def ensure_numpy(x):
    if isinstance(x, Tensor):
        return x.data
    elif isinstance(x, (np.ndarray, float, int)):
        return np.array(x) if not isinstance(x, np.ndarray) else x
    else:
        return np.array(x)

def _unbroadcast(grad, shape):
    # Sum out broadcasted dimensions so grad matches `shape`.
    if grad.shape == shape:
        return grad
    # Sum trailing dimensions introduced by broadcasting
    # Example: grad (2,3,4) -> shape (3,4) -> need to sum axis 0
    while grad.ndim > len(shape):
        grad = grad.sum(axis=0)
    # For same ndim, sum axes where shape==1 but grad>1
    for i, (g_dim, s_dim) in enumerate(zip(grad.shape, shape)):
        if s_dim == 1 and g_dim != 1:
            grad = grad.sum(axis=i, keepdims=True)
    return grad.reshape(shape)

class Tensor:
    def __init__(self, data, requires_grad=False, _children=(), _op=''):
        if isinstance(data, (list, tuple)):
            data = np.array(data)
        elif isinstance(data, Tensor):
            data = data.data.copy()
        elif not isinstance(data, np.ndarray):
            data = np.array(data)
        self.data = data.astype(float)
        self.requires_grad = requires_grad
        self.grad = None
        self._backward = lambda: None
        self._prev = set(_children)
        self._op = _op  # for graph viz/debug
        self.shape = self.data.shape

    # Basic properties
    @property
    def ndim(self):
        return self.data.ndim

    @property
    def dim(self):
        return self.data.ndim

    @property
    def T(self):
        out = Tensor(self.data.T, requires_grad=self.requires_grad, _children=(self,), _op='T')
        def _backward():
            if self.requires_grad:
                if out.grad is None:
                    return
                g = out.grad.T
                self.grad = g if self.grad is None else self.grad + g
        out._backward = _backward
        return out

    def reshape(self, *shape):
        new_shape = shape if len(shape) > 1 else shape[0] if isinstance(shape[0], (tuple, list)) else shape
        out = Tensor(self.data.reshape(new_shape), requires_grad=self.requires_grad, _children=(self,), _op='reshape')
        old_shape = self.shape
        def _backward():
            if self.requires_grad:
                if out.grad is None:
                    return
                g = out.grad.reshape(old_shape)
                self.grad = g if self.grad is None else self.grad + g
        out._backward = _backward
        return out

    # Representation
    def __repr__(self):
        return f"Tensor(data={self.data}, requires_grad={self.requires_grad})"

    # Operator helpers for binary ops with broadcasting
    def _binary_op(self, other, op_name, forward, grad_self, grad_other):
        other_data = other.data if isinstance(other, Tensor) else np.array(other)
        out_data = forward(self.data, other_data)
        requires = self.requires_grad or (other.requires_grad if isinstance(other, Tensor) else False)
        out = Tensor(out_data, requires_grad=requires, _children=(self, other if isinstance(other, Tensor) else ()), _op=op_name)

        def _backward():
            if out.grad is None:
                return
            if self.requires_grad:
                g_self = grad_self(out.grad, self.data, other_data)
                g_self = _unbroadcast(g_self, self.shape)
                self.grad = g_self if self.grad is None else self.grad + g_self
            if isinstance(other, Tensor) and other.requires_grad:
                g_other = grad_other(out.grad, self.data, other_data)
                g_other = _unbroadcast(g_other, other.shape)
                other.grad = g_other if other.grad is None else other.grad + g_other
        out._backward = _backward
        return out

    # Addition
    def __add__(self, other):
        return self._binary_op(other, 'add', lambda a,b: a + b,
                               lambda gout,a,b: gout,
                               lambda gout,a,b: gout)
    __radd__ = __add__

    # Subtraction
    def __sub__(self, other):
        return self._binary_op(other, 'sub', lambda a,b: a - b,
                               lambda gout,a,b: gout,
                               lambda gout,a,b: -gout)
    def __rsub__(self, other):
        # other - self
        other_t = other if isinstance(other, Tensor) else Tensor(other)
        return other_t.__sub__(self)

    # Negation
    def __neg__(self):
        out = Tensor(-self.data, requires_grad=self.requires_grad, _children=(self,), _op='neg')
        def _backward():
            if self.requires_grad and out.grad is not None:
                g = -out.grad
                self.grad = g if self.grad is None else self.grad + g
        out._backward = _backward
        return out

    # Multiplication
    def __mul__(self, other):
        return self._binary_op(other, 'mul', lambda a,b: a * b,
                               lambda gout,a,b: gout * b,
                               lambda gout,a,b: gout * a)
    __rmul__ = __mul__

    # Division
    def __truediv__(self, other):
        return self._binary_op(other, 'div', lambda a,b: a / b,
                               lambda gout,a,b: gout / b,
                               lambda gout,a,b: -gout * a / (b**2))
    def __rtruediv__(self, other):
        other_t = other if isinstance(other, Tensor) else Tensor(other)
        return other_t.__truediv__(self)

    # Power
    def __pow__(self, power):
        if isinstance(power, Tensor):
            children = (self, power)
        else:
            children = (self,)
        power_val = power.data if isinstance(power, Tensor) else power
        out = Tensor(self.data ** power_val, requires_grad=self.requires_grad or (isinstance(power, Tensor) and power.requires_grad), _children=children, _op='pow')
        def _backward():
            if out.grad is None:
                return
            if self.requires_grad:
                g_self = out.grad * (power_val * (self.data ** (power_val - 1)))
                g_self = _unbroadcast(g_self, self.shape)
                self.grad = g_self if self.grad is None else self.grad + g_self
            if isinstance(power, Tensor) and power.requires_grad:
                # d/dp a^p = a^p * log(a)
                g_p = out.grad * (self.data ** power_val) * np.log(self.data + 1e-20)
                g_p = _unbroadcast(g_p, power.shape)
                power.grad = g_p if power.grad is None else power.grad + g_p
        out._backward = _backward
        return out

    # sqrt
    def sqrt(self):
        return self.__pow__(0.5)

    # exp, log
    def exp(self):
        out = Tensor(np.exp(self.data), requires_grad=self.requires_grad, _children=(self,), _op='exp')
        def _backward():
            if self.requires_grad and out.grad is not None:
                g = out.grad * out.data  # exp(x) derivative is exp(x)
                self.grad = g if self.grad is None else self.grad + g
        out._backward = _backward
        return out

    def log(self):
        out = Tensor(np.log(self.data + 1e-20), requires_grad=self.requires_grad, _children=(self,), _op='log')
        def _backward():
            if self.requires_grad and out.grad is not None:
                g = out.grad / (self.data + 1e-20)
                self.grad = g if self.grad is None else self.grad + g
        out._backward = _backward
        return out

    # Matrix multiplication (supports batched matmul via np.matmul)
    def __matmul__(self, other):
        other_data = other.data if isinstance(other, Tensor) else np.array(other)
        out_data = self.data @ other_data
        requires = self.requires_grad or (other.requires_grad if isinstance(other, Tensor) else False)
        out = Tensor(out_data, requires_grad=requires, _children=(self, other if isinstance(other, Tensor) else ()), _op='matmul')
        def _backward():
            if out.grad is None:
                return
            g = out.grad
            if self.requires_grad:
                # grad wrt a: g @ b.T (with proper broadcasting)
                grad_a = g @ np.swapaxes(other_data, -1, -2)
                grad_a = _unbroadcast(grad_a, self.shape)
                self.grad = grad_a if self.grad is None else self.grad + grad_a
            if isinstance(other, Tensor) and other.requires_grad:
                grad_b = np.swapaxes(self.data, -1, -2) @ g
                grad_b = _unbroadcast(grad_b, other.shape)
                other.grad = grad_b if other.grad is None else other.grad + grad_b
        out._backward = _backward
        return out

    # Sum with axis/dim support
    def sum(self, axis=None, keepdims=False):
        out_data = self.data.sum(axis=axis, keepdims=keepdims)
        out = Tensor(out_data, requires_grad=self.requires_grad, _children=(self,), _op='sum')
        def _backward():
            if self.requires_grad and out.grad is not None:
                grad = out.grad
                if not keepdims and axis is not None:
                    # need to reshape grad to have singleton dims where sum occurred
                    if isinstance(axis, int):
                        axes = (axis,)
                    else:
                        axes = tuple(axis)
                    shape = list(self.shape)
                    for ax in axes:
                        shape[ax] = 1
                    grad = grad.reshape(shape)
                # broadcast to self.shape
                grad = np.broadcast_to(grad, self.shape)
                self.grad = grad if self.grad is None else self.grad + grad
        out._backward = _backward
        return out

    # mean over axis(s)
    def mean(self, axis=None, keepdims=False):
        if axis is None:
            denom = self.data.size
        else:
            if isinstance(axis, int):
                axes = (axis,)
            else:
                axes = tuple(axis)
            denom = 1
            for ax in axes:
                denom *= self.shape[ax]
        out_data = self.data.mean(axis=axis, keepdims=keepdims)
        out = Tensor(out_data, requires_grad=self.requires_grad, _children=(self,), _op='mean')
        def _backward():
            if self.requires_grad and out.grad is not None:
                grad = out.grad
                if not keepdims and axis is not None:
                    if isinstance(axis, int):
                        axes = (axis,)
                    else:
                        axes = tuple(axis)
                    shape = list(self.shape)
                    for ax in axes:
                        shape[ax] = 1
                    grad = grad.reshape(shape)
                grad = np.broadcast_to(grad, self.shape) / denom
                self.grad = grad if self.grad is None else self.grad + grad
        out._backward = _backward
        return out

    # gather with dim/axis
    # index is expected to be integer Tensor with indices along `dim`
    def gather(self, dim, index):
        idx = index.data.astype(int) if isinstance(index, Tensor) else np.array(index).astype(int)
        out_data = np.take_along_axis(self.data, idx, axis=dim)
        requires = self.requires_grad or (index.requires_grad if isinstance(index, Tensor) else False)
        children = (self, index) if isinstance(index, Tensor) else (self,)

        out = Tensor(
            out_data,
            requires_grad=requires,
            _children=children,
            _op='gather'
        )

        def _backward():
            if out.grad is None:
                return
            if self.requires_grad:
                # scatter out.grad back into shape of self.data using idx
                grad_self = np.zeros_like(self.data)
                # Build full index arrays for np.add.at
                # Create grid of indices for all axes
                # idx has same shape as out.grad
                # We want to index at positions where along axis dim we put idx, and other axes are broadcasted from np.indices
                grid = np.ogrid[tuple(slice(0, s) for s in out.grad.shape)]
                # Convert to list of index arrays for advanced indexing
                indexer = []
                for axis in range(self.data.ndim):
                    if axis == dim:
                        # need idx entries
                        indexer.append(idx)
                    else:
                        # expand index grid for this axis to shape of idx
                        # find corresponding axis in out.grad: mapping depends on shapes; assume shapes match except along dim
                        # We'll create an index array by broadcasting arange of that axis length to out.grad shape
                        length = self.data.shape[axis]
                        shape = [1]*out.grad.ndim
                        # Determine where this axis falls in out.grad shape:
                        # We assume out.grad shape equals idx.shape
                        # We need to get an array of indices 0..length-1 placed into the proper axis location so it broadcasts
                        arr = np.arange(length).reshape([-1 if i==0 else 1 for i in range(1)])  # placeholder (won't use)
                        # A simpler approach: construct indexer by using np.indices of out.grad.shape
                        pass
                # Simpler scatter using loop over flattened entries (vectorized enough for reasonable sizes)
                flat_idx = idx.ravel()
                flat_grad = out.grad.ravel()
                # Compute multi-index for each element in flat_idx
                # To find the coordinates in self.data corresponding to each element in out.grad, we need coordinates for other axes.
                # Use np.indices to get coordinate grids for out.grad's shape
                coords = np.indices(out.grad.shape)
                # coords is shape (ndim_out, *out.grad.shape). For each axis of out.grad, coords[ax] gives indices along that axis.
                # We must map coords axes to self.data axes. When self.data and out.grad shapes differ only on dim,
                # assume other axes match sizes. So coords along axis j correspond to same axis j in self.data except at dim where we use flat_idx.
                # Build tuple of arrays for destination indices:
                dest_idx = []
                for axis in range(self.data.ndim):
                    if axis == dim:
                        dest_idx.append(flat_idx.reshape(out.grad.shape))
                    else:
                        dest_idx.append(coords[axis])
                # Use numpy.add.at with tuple(dest_idx)
                np.add.at(grad_self, tuple(dest_idx), out.grad)
                self.grad = grad_self if self.grad is None else self.grad + grad_self
            # Note: index (indices) is usually integer, not differentiable; we skip gradient for index.
        out._backward = _backward
        return out

    # Sum alias
    def sum_all(self):
        return self.sum(axis=None)

    # backward engine
    def backward(self, grad=None):
        if not self.requires_grad:
            return
        if grad is None:
            grad = np.ones_like(self.data)
        self.grad = grad if self.grad is None else self.grad + grad
        topo = []
        visited = set()
        def build(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    if not isinstance(child, Tensor):
                        raise TypeError(f"Graph child is not a Tensor, got {type(child)}")
                    build(child)
                topo.append(v)

        build(self)
        for node in reversed(topo):
            node._backward()



In [53]:


# 4) power, exp, log, sqrt
t = Tensor(np.array([[2.,3.],[4.,5.]]), requires_grad=True)
z = (t ** 2).exp().log() + t.sqrt()
res = z.sum()
print(t)
print(z)
res.backward()
print("\nTest4: math ops")
print("t.grad:\n", t.grad)

# Note: gather test (basic)
src = Tensor(np.array([[10,20,30],[40,50,60]]).astype(float), requires_grad=True)
idx = Tensor(np.array([[2,0],[1,2]]), requires_grad=False)
g = src.gather(1, idx)  # gather along axis 1
L = g.sum()
L.backward()
print("\nTest5: gather")
print("g.data:\n", g.data)
print("src.grad:\n", src.grad)

# End of demonstration.



Tensor(data=[[2. 3.]
 [4. 5.]], requires_grad=True)
Tensor(data=[[ 5.41421356 10.73205081]
 [18.         27.23606798]], requires_grad=True)

Test4: math ops
t.grad:
 [[ 4.35355339  6.28867513]
 [ 8.25       10.2236068 ]]

Test5: gather
g.data:
 [[30. 10.]
 [50. 60.]]
src.grad:
 [[1. 0. 1.]
 [0. 1. 1.]]


In [54]:
a = Tensor(np.array([[1, 2], [3, 4]]), requires_grad=True)
b = Tensor(np.random.randint(0,4,size = (2,3,2)), requires_grad=True)

In [55]:
c = b@a

In [56]:
c.backward()

In [57]:
c.grad, b.grad, a.grad

(array([[[1., 1.],
         [1., 1.],
         [1., 1.]],
 
        [[1., 1.],
         [1., 1.],
         [1., 1.]]]),
 array([[[3., 7.],
         [3., 7.],
         [3., 7.]],
 
        [[3., 7.],
         [3., 7.],
         [3., 7.]]]),
 array([[ 9.,  9.],
        [11., 11.]]))

In [58]:
x = Tensor(np.random.randint(0,10,size= (2,3,3)), requires_grad=True)


In [59]:
x

Tensor(data=[[[2. 8. 4.]
  [2. 3. 8.]
  [0. 4. 3.]]

 [[2. 0. 9.]
  [8. 2. 7.]
  [8. 0. 2.]]], requires_grad=True)

In [60]:
x.shape, x.sum(axis=1)

((2, 3, 3),
 Tensor(data=[[ 4. 15. 15.]
  [18.  2. 18.]], requires_grad=True))

In [61]:
y = Tensor([1,2,3,4,5,6], requires_grad=True)

In [62]:
z = y.gather(dim = 0, index = [2])

In [63]:
f = z.exp()

In [64]:
f.backward()